# **LOAD DATASET AND LIBRARY AND SPLIT FOR TRAIN AND VALIDATION**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Helsinki-NLP/opus_books", "en-fr")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.2)
train_data = dataset['train']
val_data = dataset['test']

In [ ]:
val_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 25417
})

In [ ]:
train_data[1]

{'id': '67351',
 'translation': {'en': 'Maitre Lecanu did not know in the least.',
  'fr': "M. Lecanu l'ignorait parfaitement."}}

# **TOKENIZE THE DATASET**

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

def tokenize(sentences):
  input = [x['en'] for x in sentences['translation']]
  translated = [x['fr'] for x in sentences['translation']]

  model_input = tokenizer(input, truncation = True, padding = True, max_length = 128, text_target=translated)


  return model_input

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenized_datasets = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train'][502]

{'id': '90192',
 'translation': {'en': '"But," Conseil asked, "could master tell me the cause of this effect, because I presume this water hasn\'t really changed into milk!"',
  'fr': "-- Mais, demanda Conseil, monsieur peut-il m'apprendre quelle cause produit un pareil effet, car cette eau ne s'est pas changée en lait, je suppose !"},
 'input_ids': [58,
  16213,
  1988,
  184,
  2308,
  2,
  58,
  1071,
  22839,
  6469,
  1860,
  143,
  4,
  910,
  7,
  67,
  1507,
  2,
  568,
  47,
  48740,
  67,
  803,
  18979,
  6,
  75,
  1453,
  4971,
  208,
  7405,
  8888,
  0,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  59513,
  

# **FINE-TUNING**

In [ ]:
from transformers import MarianMTModel

model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors='tf')

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-1249927301.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

trainer.save_model("./en-fr-translation-model")
tokenizer.save_pretrained("./en-fr-translation-model")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ameybhagat589 (ameybhagat589-nitk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.626600
1000,0.487900
1500,0.477500
2000,0.467300
2500,0.465400
3000,0.455100
3500,0.453300
4000,0.448600
4500,0.448600
5000,0.437400


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./en-fr-translation-model/tokenizer_config.json',
 './en-fr-translation-model/special_tokens_map.json',
 './en-fr-translation-model/vocab.json',
 './en-fr-translation-model/source.spm',
 './en-fr-translation-model/target.spm',
 './en-fr-translation-model/added_tokens.json')

# **TEST THE MODEL**

In [ ]:
text = "recruitments for web club will be on 26th october."
inputs = tokenizer(text, return_tensors="pt", padding=True).to(model.device)
translated_tokens = model.generate(**inputs)
print(tokenizer.decode(translated_tokens[0], skip_special_tokens=True))


Les recrutements pour le web-club auront lieu le 26 octobre.


In [ ]:

pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
import evaluate
bleu = evaluate.load("bleu")
predictions = ["Les recrutements pour le web-club auront lieu le 26 octobre."]
references = [["les recrutements pour le web club se feront le 26 octobre."]]


In [ ]:
results = bleu.compute(predictions=predictions, references=references)
print(results)


{'bleu': 0.3098380229804167, 'precisions': [0.6363636363636364, 0.5, 0.3333333333333333, 0.125], 'brevity_penalty': 0.9131007162822622, 'length_ratio': 0.9166666666666666, 'translation_length': 11, 'reference_length': 12}
